In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_27.csv')
pred2 = dd.read_csv('test_pred_28.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 34.2 s, sys: 3.15 s, total: 37.3 s
Wall time: 9.59 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 153 ms, sys: 297 ms, total: 450 ms
Wall time: 448 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 391 µs, sys: 44 µs, total: 435 µs
Wall time: 441 µs


In [7]:
coef_lgb = 0.9
coef_mlp = 0.1

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 328 ms, sys: 583 ms, total: 911 ms
Wall time: 910 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000292,0.341586,0.000496,0.216301,0.137334,0.000558,0.130544,0.000240,0.002394,0.021009,0.002973,0.145043,0.000372,0.000855,0.247338
3492886,130787971,0.000214,0.010632,0.000286,0.044801,0.181098,0.000326,0.019420,0.391704,0.000534,0.024561,0.000976,0.322122,0.000318,0.003009,0.239638
3492887,130787974,0.000660,0.021262,0.000164,0.886955,0.029055,0.000351,0.047300,0.000059,0.000185,0.001050,0.000492,0.010279,0.000225,0.001964,0.079695
3492888,130788053,0.000085,0.690437,0.000170,0.143561,0.004180,0.000170,0.010292,0.112988,0.000525,0.000336,0.000206,0.036079,0.000174,0.000796,0.175940
3492889,130788054,0.000258,0.090227,0.000383,0.391595,0.231864,0.000571,0.058434,0.000135,0.000752,0.020261,0.001016,0.202869,0.000771,0.000864,0.239675


In [13]:
%%time
test_ensemble.to_csv('test_pred_31.csv',index=False)

CPU times: user 1min 30s, sys: 1.21 s, total: 1min 31s
Wall time: 1min 31s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [15]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_31.csv -m "Message"

100%|█████████████████████████████████████| 1.06G/1.06G [2:47:42<00:00, 113kB/s]
Successfully submitted to PLAsTiCC Astronomical Classification